In [3]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, models, transforms
from torch.utils.data import DataLoader
import os

# Definir as transformações de dados para o treino e validação
data_transforms = {
    'train': transforms.Compose([
        transforms.Resize((224, 224)),  # Redimensiona para o tamanho de entrada da ResNet
        transforms.ToTensor(),  # Converte a imagem para tensor
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),  # Normalização com valores do ImageNet
    ]),
    'val': transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
    ]),
}

# Diretórios dos dados
data_dir = r"01.Dados Treino/Classificacao"

# Carregar os datasets de treino e validação
image_datasets = {x: datasets.ImageFolder(os.path.join(data_dir, x), data_transforms[x])
                  for x in ['train', 'val']}

# Criar dataloaders para treinamento e validação
dataloaders = {x: DataLoader(image_datasets[x], batch_size=32, shuffle=True, num_workers=4)
               for x in ['train', 'val']}

# Carregar o modelo ResNet18 pré-treinado
model = models.resnet18(pretrained=True)

# Substituir a última camada totalmente conectada para corresponder ao número de classes (2 classes: Normal e Quebrado)
num_ftrs = model.fc.in_features
model.fc = nn.Linear(num_ftrs, 2)

# Mover o modelo para GPU, se disponível
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)

# Definir a função de perda (cross-entropy) e o otimizador (Adam)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Função para treinar o modelo
def train_model(model, dataloaders, criterion, optimizer, num_epochs=10):
    best_model_classif = model.state_dict()
    best_acc = 0.0

    for epoch in range(num_epochs):
        print(f'Epoch {epoch + 1}/{num_epochs}')
        print('-' * 10)

        # Cada época tem uma fase de treinamento e uma de validação
        for phase in ['train', 'val']:
            model.train() if phase == 'train' else model.eval()

            running_loss = 0.0
            running_corrects = 0

            # Iterar sobre os dados
            for inputs, labels in dataloaders[phase]:
                inputs = inputs.to(device)
                labels = labels.to(device)

                # Zero os gradientes dos parâmetros
                optimizer.zero_grad()

                # Forward
                with torch.set_grad_enabled(phase == 'train'):
                    outputs = model(inputs)
                    loss = criterion(outputs, labels)

                    # Backward + otimizador
                    if phase == 'train':
                        loss.backward()
                        optimizer.step()

                # Estatísticas
                _, preds = torch.max(outputs, 1)
                running_loss += loss.item() * inputs.size(0)
                running_corrects += torch.sum(preds == labels.data)

            epoch_loss = running_loss / len(dataloaders[phase].dataset)
            epoch_acc = running_corrects.double() / len(dataloaders[phase].dataset)

            print(f'{phase} Loss: {epoch_loss:.4f} Acc: {epoch_acc:.4f}')

            # Salvar o modelo ao final de cada época
            model_save_path = f'best_model_classif.pth'
            torch.save(model.state_dict(), model_save_path)
            print(f'Modelo salvo em {model_save_path}')

            # Copiar o modelo com melhor acurácia
            if phase == 'val' and epoch_acc > best_acc:
                best_acc = epoch_acc
                best_model_classif = model.state_dict()

        print()

    print(f'Best val Acc: {best_acc:.4f}')

    # Carregar os pesos do melhor modelo
    model.load_state_dict(best_model_classif)
    return model

# Treinar o modelo
model = train_model(model, dataloaders, criterion, optimizer, num_epochs=10)

Epoch 1/10
----------
train Loss: 0.2038 Acc: 0.9247
Modelo salvo em best_model_classif.pth
val Loss: 2.9773 Acc: 0.6400
Modelo salvo em best_model_classif.pth

Epoch 2/10
----------
train Loss: 0.0356 Acc: 0.9892
Modelo salvo em best_model_classif.pth
val Loss: 0.2521 Acc: 0.9200
Modelo salvo em best_model_classif.pth

Epoch 3/10
----------
train Loss: 0.0468 Acc: 0.9928
Modelo salvo em best_model_classif.pth
val Loss: 0.3335 Acc: 0.8800
Modelo salvo em best_model_classif.pth

Epoch 4/10
----------
train Loss: 0.0733 Acc: 0.9767
Modelo salvo em best_model_classif.pth
val Loss: 0.1014 Acc: 0.9200
Modelo salvo em best_model_classif.pth

Epoch 5/10
----------
train Loss: 0.0084 Acc: 1.0000
Modelo salvo em best_model_classif.pth
val Loss: 0.1042 Acc: 0.9600
Modelo salvo em best_model_classif.pth

Epoch 6/10
----------
train Loss: 0.0044 Acc: 0.9982
Modelo salvo em best_model_classif.pth
val Loss: 0.0111 Acc: 1.0000
Modelo salvo em best_model_classif.pth

Epoch 7/10
----------
train Loss: 